# Prediction intervals

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/etna-team/etna/master?filepath=examples/306-prediction_intervals.ipynb)

This notebook contains overview of prediction intervals functionality in ETNA library.

**Table of contents**

* [Loading and preparing data](#chapter1)
* [Estimating intervals using builtin method](#chapter2)
    * [Accessing prediction intervals in `TSDataset`](#chapter2_1)
    * [Computing interval metrics](#chapter2_2)
* [Estimating prediction intervals using `experimental.prediction_intervals` module](#chapter3)
    * [`NaiveVariancePredictionIntervals`](#chapter3_1)
    * [`ConformalPredictionIntervals`](#chapter3_2)
    * [`EmpiricalPredictionIntervals`](#chapter3_3)
* [Custom prediction interval method](#chapter4)
    * [Non-parametric method](#chapter4_1)
    * [Estimating historical residuals](#chapter4_2)

In [ ]:
import warnings

import numpy as np
import pandas as pd

from etna.analysis.forecast import plot_forecast
from etna.datasets import TSDataset
from etna.metrics import Coverage
from etna.metrics import Width
from etna.models import CatBoostMultiSegmentModel
from etna.pipeline import Pipeline
from etna.transforms import DateFlagsTransform
from etna.transforms import LagTransform
from etna.transforms import LinearTrendTransform
from etna.transforms import LogTransform
from etna.transforms import MeanTransform
from etna.transforms import SegmentEncoderTransform

warnings.filterwarnings("ignore")

In [ ]:
HORIZON = 30

## Loading and preparing data <a class="anchor" id="chapter1"></a>


Consider the dataset `data/example_dataset.csv`.

This data will be used to show how prediction intervals could be estimated and accessed in ETNA library.

First step is to load data and convert it to the `TSDataset`.

In [ ]:
df = pd.read_csv("data/example_dataset.csv")
df = TSDataset.to_dataset(df=df)
ts = TSDataset(df=df, freq="D")

ts

In [ ]:
ts.plot()

Here we have four segments in the dataset. All segments have seasonalities, and some of them show signs of trend.
Note that segment C contains obvious outlier, that may affect quality of estimated intervals.

With the next step we split our dataset into two parts: train and test. Test part will be used as a hold out dataset for metrics
computation and result analysis.

In [ ]:
train_ts, test_ts = ts.train_test_split(test_size=HORIZON)

## Estimating intervals using builtin method <a class="anchor" id="chapter2"></a>

Currently there are several types of prediction intervals in the library:
1. Quantiles estimates
2. Arbitrary interval borders, that tends to provide desired coverage

Methods, implemented in the library, use univariate distribution to estimate quantiles at each timestamp in the horizon. There is possibility to treat all timestamps in the horizon jointly
and threat horizon as multivariate random variable to estimate quatiles. Extension of current method pool will be discussed in the last section of this notebook.

So there are some naming convention to achieve distinction between two types of intervals. Borders that approximate quantiles named in the following format `{target_{q:.4g}}`, where `q`
is corresponding quantile level. And there no particular rules for the arbitrary borders. But it is implementation responsibility to name them appropriately.

Before estimating prediction intervals we need to fit a model. Here `CatBoostMultiSegmentModel` is used with lag and date features.
This model requires computed features, so we add corresponding transforms to the pipeline.

In [ ]:
log = LogTransform(in_column="target")
trend = LinearTrendTransform(in_column="target")
seg = SegmentEncoderTransform()
lags = LagTransform(in_column="target", lags=list(range(HORIZON, 20 + HORIZON)), out_column="lag")
date_flags = DateFlagsTransform(
    day_number_in_week=True,
    day_number_in_month=True,
    week_number_in_month=True,
    week_number_in_year=True,
    month_number_in_year=True,
    year_number=True,
    is_weekend=True,
)
mean = MeanTransform(in_column=f"lag_{HORIZON}", window=30)

transforms = [
    # log,
    # trend,
    lags,
    date_flags,
    seg,
    # mean
]

In [ ]:
model = CatBoostMultiSegmentModel()

In [ ]:
pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

pipeline.fit(ts=train_ts);

After pipeline is defined and fitted we are able to estimate prediction intervals with the default method. To do so
set `prediction_interval=True` parameter of `forecast` method.

This prediction intervals method is based on residual variance estimation and $z$-scores. Variance estimation is done via running
historical backtest on non-overlapping folds. Number of folds controlled via `n_folds` parameter.

In [ ]:
forecast = pipeline.forecast(ts=train_ts, prediction_interval=True, n_folds=7)
forecast

Here we have point forecast for full horizon alongside with estimated prediction interval for each segment.

Section below describes how one can perform manipulations with intervals in the dataset of forecasts.

### Accessing prediction intervals in `TSDataset` <a class="anchor" id="chapter2_1"></a>

Column names for the estimated prediction intervals can be obtained using `TSDataset.prediction_intervals_names` property.

In [ ]:
forecast.prediction_intervals_names

Here segment names are omitted, because they share interval estimation method. So column names identical for all the segments.

Dataframe with prediction intervals only for each segment can be obtained by using `TSDataset.get_prediction_intervals()` method.

Here we save such dataframe to the separate object to use it later.

In [ ]:
prediction_intervals = forecast.get_prediction_intervals()
prediction_intervals

If estimated intervals are no longer needed or there is a necessity to remove prediction intervals from the dataset use
`TSDataset.drop_prediction_intervals()` method.

Once we removed intervals, we can check that they are no longer presented by looking at stored names.

In [ ]:
forecast.drop_prediction_intervals()
forecast.prediction_intervals_names

Here we see that property contains empty tuple now. It is indication that no intervals are registered.

In [ ]:
forecast.get_prediction_intervals()

Calling `TSDataset.get_prediction_intervals()` in such case will return `None`.

There is a possibility to add existing prediction intervals to the dataset. To do so one should use
 `TSDataset.add_prediction_intervals()` method.

There are a couple requirements when adding existing intervals to the dataset.
1. There are should be no intervals in the dataset. This could be checked via `prediction_intervals_names` property.
2. Dataframe with intervals should be in ETNA wide format.
3. All segments should be matched between dataset and intervals dataframe
4. Interval borders sets should match for all the segments.


In [ ]:
forecast.add_prediction_intervals(prediction_intervals_df=prediction_intervals)
forecast.prediction_intervals_names

In [ ]:
forecast

Here we called `prediction_intervals_names` to make sure that intervals added correctly and printed out resulting dataset.

Results visualisation could be done using `plot_forecast` function. Setting parameter `prediction_intervals=True` will
enable plotting estimated prediction intervals.

In [ ]:
plot_forecast(forecast, test_ts, train_ts, prediction_intervals=True, n_train_samples=30)

### Computing interval metrics <a class="anchor" id="chapter2_2"></a>

There are a couple of metrics in the library that can help to estimate quality if computed prediction intervals:
* `Coverage` - percentage of points in the horizon covered by the interval
* `Width` - mean width of the prediction interval on full horizon.

This metrics require initialization. To specify which interval to use provide border names by setting
`lower_name` and `upper_name` parameters. After initialization this metrics will try to find specified borders in
 the dataset with predicted values. If provided names not found corresponding error will be raised.

Here we wrap metrics estimation in one function.

In [ ]:
def interval_metrics(test_ts, forecast):
    lower_name, upper_name = forecast.prediction_intervals_names

    coverage = Coverage(lower_name=lower_name, upper_name=upper_name)(test_ts, forecast)
    width = Width(lower_name=lower_name, upper_name=upper_name)(test_ts, forecast)

    return coverage, width

In [ ]:
coverage, width = interval_metrics(test_ts=test_ts, forecast=forecast)

In [ ]:
coverage

In [ ]:
width

## Estimating prediction intervals using `experimental.prediction_intervals` module <a class="anchor" id="chapter3"></a>

ETNA library provides several alternative methods for prediction intervals estimation. All necessary functionality is at
`etna.experimental.prediction_intervals` module.

This section covers currently implemented methods. Also module provides possibility to easily extend methods list
by implementing custom one. This topic will be discussed in the next section.

Prediction intervals functionality is implemented via wrapper classes for the ETNA pipelines. While initialization such
methods require pipeline instance and necessary hyperparameters. Provided pipeline can be fitted before or after wrapping
with the intervals estimation method.

### `NaiveVariancePredictionIntervals` <a class="anchor" id="chapter3_1"></a>
This method estimate prediction quantiles using the following algorithm:

1. Compute the residuals matrix $r_{it} = \hat y_{it} - y_{it}$ using k-fold backtest, where $i$ is fold index.

2. Estimate variance for each step in the prediction horizon $v_t = \frac{1}{k} \sum_{i = 1}^k r_{it}^2$.

3. Use $z$-scores and estimated variance to compute corresponding quantiles.


Desired quantiles levels for the prediction interval can be set via `quantiles` of the `forecast` method.

In [ ]:
from etna.experimental.prediction_intervals import NaiveVariancePredictionIntervals

pipeline = NaiveVariancePredictionIntervals(pipeline=pipeline)

forecast = pipeline.forecast(quantiles=(0.025, 0.975), prediction_interval=True, n_folds=40)

forecast

In [ ]:
plot_forecast(forecast, test_ts, train_ts, prediction_intervals=True, n_train_samples=30)

In [ ]:
coverage, width = interval_metrics(test_ts=test_ts, forecast=forecast)

In [ ]:
coverage

In [ ]:
width

### `ConformalPredictionIntervals` <a class="anchor" id="chapter3_2"></a>

Estimates conformal prediction intervals:

1. Compute matrix of absolute residuals  $r_{it} = |\hat y_{it} - y_{it}|$ using k-fold historical backtest, where $i$ is fold index.

2. Estimate corresponding quantiles levels using the provided coverage (e.g. apply Bonferroni correction).

3. Estimate quantiles for each horizon step separately using computed absolute residuals and levels.


**Note**: this method estimates arbitrary interval bounds that tends to provide given coverage rate.
So this method ignores `quantiles` parameter of `forecast` method.

Coverage rate and correction option should be set at method initialization step.

In [ ]:
from etna.experimental.prediction_intervals import ConformalPredictionIntervals

pipeline = ConformalPredictionIntervals(pipeline=pipeline, coverage=0.95, bonferroni_correction=True)

forecast = pipeline.forecast(prediction_interval=True, n_folds=40)

forecast

In [ ]:
plot_forecast(forecast, test_ts, train_ts, prediction_intervals=True, n_train_samples=30)

In [ ]:
coverage, width = interval_metrics(test_ts=test_ts, forecast=forecast)

In [ ]:
coverage

In [ ]:
width

### `EmpiricalPredictionIntervals` <a class="anchor" id="chapter3_3"></a>

Estimates prediction intervals via historical residuals:

1. Compute matrix of residuals  $r_{it} = |\hat y_{it} - y_{it}|$ using k-fold backtest, where $i$ is fold index.

2. Estimate quantiles levels, that satisfy the provided coverage, for the corresponding residuals distributions.

3. Estimate quantiles for each timestamp using computed residuals and levels.


**Note**: this method estimates arbitrary interval bounds that tends to provide given coverage rate.
So this method ignores `quantiles` parameter of `forecast` method.

Coverage rate and correction option should be set at method initialization step.

In [ ]:
from etna.experimental.prediction_intervals import EmpiricalPredictionIntervals

pipeline = EmpiricalPredictionIntervals(pipeline=pipeline)

forecast = pipeline.forecast(prediction_interval=True, n_folds=40)

forecast

In [ ]:
plot_forecast(forecast, test_ts, train_ts, prediction_intervals=True, n_train_samples=30)

In [ ]:
coverage, width = interval_metrics(test_ts=test_ts, forecast=forecast)

In [ ]:
coverage

In [ ]:
width

### Ensemble example

## Custom prediction interval method <a class="anchor" id="chapter4"></a>

There is a possibility in the library to extend set of prediction intervals methods by implementing desired algorithm.
This section demonstrates may how it can be done. Examples of interface and utilities usage provided as well

`BasePredictionIntervals` - base class for prediction intervals methods.

This class implements a wrapper interface for pipelines and ensembles that provides the ability to
estimate prediction intervals. So it requires a pipeline instance to be provided to `__init__` method to do proper initialization.

To add a particular method for pipelines, one must inherit from this class and provide an implementation for the
abstract method ``_forecast_prediction_interval``. This method should estimate and store prediction
intervals for out-of-sample forecasts.

**Limitations**
In-sample prediction is not supported by default and will raise a corresponding error while attempting to do so.
This functionality could be implemented if needed by overriding ``_predict`` method, which is responsible
for building an in-sample point forecast and adding prediction intervals.

### Non-parametric method <a class="anchor" id="chapter4_1"></a>

Example below demonstrates how interval method could be implemented.

Consider `ConstantWidthInterval` which simply adds constant `width` to point forecast. Here `width` is hyperparameter
that will be set on method initialization step.

In [ ]:
from typing import Sequence

from etna.experimental.prediction_intervals import BasePredictionIntervals
from etna.pipeline import BasePipeline


class ConstantWidthInterval(BasePredictionIntervals):
    def __init__(self, pipeline: BasePipeline, interval_width: float):
        assert interval_width > 0

        self.interval_width = interval_width
        super().__init__(pipeline=pipeline)

    def _forecast_prediction_interval(
        self, ts: TSDataset, predictions: TSDataset, quantiles: Sequence[float], n_folds: int
    ) -> TSDataset:
        predicted_target = predictions[..., "target"]

        lower_border = predicted_target - self.interval_width / 2
        upper_border = predicted_target + self.interval_width / 2

        upper_border.rename({"target": "target_upper"}, inplace=True, axis=1)
        lower_border.rename({"target": "target_lower"}, inplace=True, axis=1)

        predictions.add_prediction_intervals(prediction_intervals_df=pd.concat([lower_border, upper_border], axis=1))
        return predictions

In [ ]:
pipeline = ConstantWidthInterval(pipeline=pipeline, interval_width=150)

forecast = pipeline.forecast(prediction_interval=True, n_folds=40)

forecast

In [ ]:
plot_forecast(forecast, test_ts, train_ts, prediction_intervals=True, n_train_samples=30)

In [ ]:
coverage, width = interval_metrics(test_ts=test_ts, forecast=forecast)

In [ ]:
coverage

In [ ]:
width

### Estimating historical residuals <a class="anchor" id="chapter4_2"></a>

Some prediction intervals methods require doing forecast on historical data. This could be done by
using pipelines `get_historical_forecasts` method. As `BasePredictionIntervals` wraps pipelines this method implemented here as well.

Consider `MaxAbsResidInterval` example method. It estimates intervals based on maximum absolute values of historical
residuals for each segment. So we can brake down this algorithm into following steps:
1. Estimate historical forecast by calling `get_historical_forecasts` method
2. For each `segment` estimate residuals, find maximum absolute value and add to point forecast

In [ ]:
class MaxAbsResidInterval(BasePredictionIntervals):
    def __init__(self, pipeline: BasePipeline, coverage: float = 0.95, stride: int = 1):
        assert stride > 0
        assert 0 < coverage <= 1

        self.stride = stride
        self.coverage = coverage
        super().__init__(pipeline=pipeline)

    def _forecast_prediction_interval(
        self, ts: TSDataset, predictions: TSDataset, quantiles: Sequence[float], n_folds: int
    ) -> TSDataset:
        predicted_target = predictions[..., "target"]

        lower_border = predicted_target.copy()
        upper_border = predicted_target.copy()

        fold_forecast = self.get_historical_forecasts(ts=ts, n_folds=n_folds, stride=self.stride)

        for segment in ts.segments:
            residuals = (
                ts.loc[:, pd.IndexSlice[segment, "target"]] - fold_forecast.loc[:, pd.IndexSlice[segment, "target"]]
            )
            width = np.max(np.abs(residuals))

            lower_border.loc[:, pd.IndexSlice[segment, "target"]] -= self.coverage * width / 2
            upper_border.loc[:, pd.IndexSlice[segment, "target"]] += self.coverage * width / 2

        upper_border.rename({"target": "target_upper"}, inplace=True, axis=1)
        lower_border.rename({"target": "target_lower"}, inplace=True, axis=1)

        predictions.add_prediction_intervals(prediction_intervals_df=pd.concat([lower_border, upper_border], axis=1))
        return predictions

In [ ]:
pipeline = MaxAbsResidInterval(pipeline=pipeline)

forecast = pipeline.forecast(prediction_interval=True, n_folds=5)

forecast

In [ ]:
# plot_forecast(forecast, test_ts, train_ts, prediction_intervals=True, n_train_samples=30)

In [ ]:
coverage, width = interval_metrics(test_ts=test_ts, forecast=forecast)

In [ ]:
coverage

In [ ]:
width

Obtaining historical residuals for prediction intervals estimation can be simplified by using more efficient utility function `residuals_matrices`.
This function accepts pipeline, data, parameters for backtest and estimates residuals for each segment on every fold.

**Note** that `residuals_matrices` function returns 3 dimensional array with axes sizes `(num_folds, horizon, num_segments)`.

Here we use this function to optimize proposed method. Consider `OptimizedMaxAbsResidInterval`.

In [ ]:
from etna.experimental.prediction_intervals.utils import residuals_matrices


class OptimizedMaxAbsResidInterval(BasePredictionIntervals):
    def __init__(self, pipeline: BasePipeline, coverage: float = 0.95, stride: int = 1):
        assert stride > 0
        assert 0 < coverage <= 1

        self.stride = stride
        self.coverage = coverage
        super().__init__(pipeline=pipeline)

    def _forecast_prediction_interval(
        self, ts: TSDataset, predictions: TSDataset, quantiles: Sequence[float], n_folds: int
    ) -> TSDataset:
        residuals = residuals_matrices(pipeline=self, ts=ts, n_folds=n_folds, stride=self.stride)

        predicted_target = predictions[..., "target"]

        width = np.max(np.abs(residuals), axis=(0, 1)).reshape(1, -1)

        lower_border = predicted_target - self.coverage * width / 2
        upper_border = predicted_target + self.coverage * width / 2

        upper_border.rename({"target": "target_upper"}, inplace=True, axis=1)
        lower_border.rename({"target": "target_lower"}, inplace=True, axis=1)

        predictions.add_prediction_intervals(prediction_intervals_df=pd.concat([lower_border, upper_border], axis=1))
        return predictions

In [ ]:
pipeline = OptimizedMaxAbsResidInterval(pipeline=pipeline)

forecast = pipeline.forecast(prediction_interval=True, n_folds=3)

forecast

In [ ]:
# plot_forecast(forecast, test_ts, train_ts, prediction_intervals=True, n_train_samples=30)

In [ ]:
coverage, width = interval_metrics(test_ts=test_ts, forecast=forecast)

In [ ]:
coverage

In [ ]:
width